In [31]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [32]:
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [33]:
print(mnist.train.images.shape)

(55000, 784)


In [34]:
n_hidden_1 = 256
n_hidden_2 = 128
n_classes = 10
n_input = 784

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_classes])

stddev = 0.1 
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev)),

}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes])),

}



In [35]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    return (tf.matmul(layer_2, weights['out']) + biases['out'])

In [36]:
pred = multilayer_perceptron(x, weights, biases)

# loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, 'float'))

# Initializer
init = tf.global_variables_initializer()

In [38]:
train_epochs = 50
batch_size = 100
display_step = 4
sess = tf.Session()
sess.run(init)

for epoch in range(train_epochs):
    avg_cost = 0
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost/num_batch
    if (epoch+1) % display_step ==0:
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print('Epoch:%4d  train_acc:%.8f    test_acc:%.8f    cost:%.8f'
             %(epoch, train_acc, test_acc, avg_cost))
print('Finished')        

Epoch:   3  train_acc:0.25999999    test_acc:0.22090000    cost:2.26366595
Epoch:   7  train_acc:0.31999999    test_acc:0.38139999    cost:2.22917535
Epoch:  11  train_acc:0.50999999    test_acc:0.48530000    cost:2.19085581
Epoch:  15  train_acc:0.41999999    test_acc:0.53869998    cost:2.14669636
Epoch:  19  train_acc:0.61000001    test_acc:0.57709998    cost:2.09441848
Epoch:  23  train_acc:0.64999998    test_acc:0.61189997    cost:2.03188624
Epoch:  27  train_acc:0.70999998    test_acc:0.63499999    cost:1.95763991
Epoch:  31  train_acc:0.61000001    test_acc:0.65420002    cost:1.87121493
Epoch:  35  train_acc:0.63000000    test_acc:0.67430001    cost:1.77386219
Epoch:  39  train_acc:0.66000003    test_acc:0.69639999    cost:1.66894136
Epoch:  43  train_acc:0.73000002    test_acc:0.71270001    cost:1.56100747
Epoch:  47  train_acc:0.69000000    test_acc:0.72780001    cost:1.45490382
Finished
